## Classification d'images avec un CNN classique
  

### Importation des librairies

In [ ]:
%%capture
%pip install rich wandb python-dotenv

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import os
import numpy as np
import cv2 as cv
import time
import torch
import torchvision
import tqdm
import json
import requests
import zipfile

### Sélection du processeur de calcul

In [ ]:
#del model
torch.cuda.empty_cache()

In [ ]:
!nvidia-smi
!lscpu

In [ ]:
use_cuda = torch.cuda.is_available()
if not use_cuda:
  print("WARNING: PYTORCH COULD NOT LOCATE ANY AVAILABLE CUDA DEVICE.\n\n" \
        "  ...make sure you have enabled your notebook to use a GPU!" \
        "  (Edit->Notebook Settings->Hardware Accelerator: GPU)")
else:
  print("All good, a GPU is available.")

In [ ]:
import multiprocessing
num_workers = multiprocessing.cpu_count()
print(f'----> number of workers: {num_workers}')

### Téléchargement du jeu de données



In [ ]:

def get_key_def(key, dict, default=None):
    if key not in dict:
        return default
    else:
        return dict[key]


In [ ]:
# on télécharge le jeu de données via Google Drive
import gdown
gdown.download(
        f"https://drive.google.com/uc?export=download&confirm=pbef&id=1lgOSw6PM4M7wuxJTqZpTzWbs3Fm5l_x5",
        '/content/insects_dataset.zip'
    )
!unzip -oq /content/insects_dataset.zip


### Controle de l'aspect aléatoire

https://pytorch.org/docs/stable/notes/randomness.html

https://pytorch.org/docs/stable/data.html#data-loading-randomness

In [ ]:
import random
import torch

def set_seed(seed=None, seed_torch=True):
  if seed is None:
    seed = np.random.choice(2 ** 32)
  random.seed(seed)
  np.random.seed(seed)
  torch.manual_seed(seed)
  torch.cuda.manual_seed_all(seed)
  torch.cuda.manual_seed(seed)
  torch.backends.cudnn.benchmark = False
  torch.backends.cudnn.deterministic = True
  print(f'Random seed {seed} has been set.')

def seed_worker(worker_id):
  worker_seed = torch.initial_seed() % 2**32
  np.random.seed(worker_seed)
  random.seed(worker_seed)

In [ ]:
np.random.seed(100)
print(np.random.permutation(10))

In [ ]:
SEED = 2021
set_seed(seed=SEED)
g_seed = torch.Generator() # Creates and returns a generator object that manages the state of the algorithm which produces pseudo random numbers.
g_seed.manual_seed(SEED)

### Dataset et Dataloader

In [ ]:
# remarque: puisque les images de ce jeu de données sont de tailles variées, il faut
# les découper pour qu'elles soient de taille commune afin de créer des minibatches

# petit 'pipeline' de transformations permettant de prétraiter les images...
base_transforms = torchvision.transforms.Compose([
    torchvision.transforms.Resize(256),
    torchvision.transforms.RandomCrop(224),
    torchvision.transforms.RandomHorizontalFlip(),
    torchvision.transforms.RandomVerticalFlip(),
    torchvision.transforms.ToTensor(),
    torchvision.transforms.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)) # normalisation des stats d'entrée (imagenet)
])

insects_dataset = torchvision.datasets.ImageFolder(root="/content/insects_dataset", transform=base_transforms)

class_names = ["abeille", "coccinelle_asiatique", "coccinelle_septpoints", "doryphore", "hanneton", "punaise_verte", "scarabee_japonais"]

sample_idxs = np.random.permutation(len(insects_dataset)).tolist()
train_sample_count, valid_sample_count = int(0.8*len(sample_idxs)), int(0.1*len(sample_idxs))
train_sampler = torch.utils.data.sampler.SubsetRandomSampler(sample_idxs[0:train_sample_count])
valid_sampler = torch.utils.data.sampler.SubsetRandomSampler(sample_idxs[train_sample_count:(train_sample_count+valid_sample_count)])
test_sampler = torch.utils.data.sampler.SubsetRandomSampler(sample_idxs[(train_sample_count+valid_sample_count):])

assert (len(train_sampler) + len(valid_sampler) + len(test_sampler)) == len(insects_dataset)
assert not any([idx in valid_sampler or idx in test_sampler for idx in train_sampler])
assert not any([idx in test_sampler for idx in valid_sampler])

#  nombre d'images qui peuvent être lues par chaque 'Sampler'
print(f"train samples count: {len(train_sampler)}")
print(f"valid samples count: {len(valid_sampler)}")
print(f"test samples count: {len(test_sampler)}")

batch_size = 64
train_loader = torch.utils.data.DataLoader(dataset=insects_dataset, batch_size=batch_size, sampler=train_sampler, num_workers=2, worker_init_fn=seed_worker, generator=g_seed)
valid_loader = torch.utils.data.DataLoader(dataset=insects_dataset, batch_size=batch_size, sampler=valid_sampler, num_workers=2, worker_init_fn=seed_worker, generator=g_seed)
test_loader = torch.utils.data.DataLoader(dataset=insects_dataset, batch_size=batch_size, sampler=test_sampler, num_workers=2, worker_init_fn=seed_worker, generator=g_seed)

## Visualisation des minibatch

In [ ]:
print(f"train minibatch count: {len(train_loader)}")
sample_images, sample_labels = next(iter(train_loader))
print(f"images tensor shape: {sample_images.shape}")  # BxCxHxW
print(f"labels tensor shape: {sample_labels.shape}")  # Bx1  (une étiquette par image du minibatch)
display_batch_size = min(8, batch_size)
fig = plt.figure(figsize=(18, 3))
for ax_idx in range(display_batch_size):
  ax = fig.add_subplot(1, 8, ax_idx + 1)
  ax.grid(False)
  ax.set_xticks([])
  ax.set_yticks([])
  class_name = class_names[sample_labels[ax_idx]]
  ax.set_title(class_name)
  display = sample_images[ax_idx, ...].numpy()
  display = display.transpose((1, 2, 0))  # CxHxW => HxWxC (tel que demandé par matplotlib)
  mean = np.array([0.485, 0.456, 0.406])  # nécessaire pour inverser la normalisation
  std = np.array([0.229, 0.224, 0.225])  # nécessaire pour inverser la normalisation
  display = std * display + mean  # on inverse la normalisation
  display = np.clip(display, 0, 1)  # on élimine les valeurs qui sortent de l'intervalle d'affichage
  plt.imshow(display)
plt.show()


## Définition du modèle

Models pretrained used : <a href="https://pytorch.org/docs/stable/torchvision/models.html#torchvision-models">**torchvision.models**</a>.  


## 6.3. ResNet-18 (2015)
ref : https://arxiv.org/abs/1512.03385


In [ ]:
model = torchvision.models.resnet18(pretrained=True) #to do only transfert model


In [ ]:
n_param= 0
for l, (name,param) in enumerate(model.named_parameters()):
  n_param += 1
  print(name)
print(n_param)

### Liste des couches du résau:

1. simple perceptron 

In [ ]:
model.fc = torch.nn.Linear(in_features=512, out_features=len(class_names), bias=True)
print(model)
model_init_state = model.state_dict()

In [ ]:
# On peut décider quelles couches sont mises à jour (`requires_grad = True`, par défaut) ou celles qui seront gelées (`requires_grad = False`). On peut aussi spécifier une valeur spécifique du `learning_rate` pour chacune des couches.

def freeze_layers(model, n_max= 0, learning_rate= 1e-3):
  params_to_update = []
  n_frozen, n_update= 0,0
  for l, (name,param) in enumerate(model.named_parameters()):
    if l < int(n_max): # on controle la profondeur de la mise à jour ici
      param.requires_grad = False # freeze!
      n_frozen += 1
    else:
      n_update += 1
      param.requires_grad= True
      params_to_update.append({
                      "params": param,
                      "lr": learning_rate,
                  })
    if param.requires_grad == True:
        print("\t",name)
  print(f'frozen: {n_frozen} updated: {n_update}')
  return params_to_update

params_to_update= freeze_layers(model, n_max= n_param - 2)

In [ ]:
print(params_to_update)

## 6.5. DenseNet-161

  ref: https://arxiv.org/abs/1608.06993

In [ ]:
model = torchvision.models.densenet161(pretrained=True)
model.classifier = torch.nn.Linear(in_features=model.classifier.in_features,
                                   out_features=len(class_names), bias=True)
print(model)

freeze_feature_layer_max_idx = 0  # à ajuster au besoin!
for layer_idx in range(freeze_feature_layer_max_idx):
  if layer_idx == 0:
    model.features.conv0.requires_grad = False
  elif layer_idx == 1:
    model.features.norm0.requires_grad = False
  else:
    layer_name = f"denseblock{layer_idx-1}"
    getattr(model.features, layer_name).requires_grad = False
    layer_name = f"transition{layer_idx-1}"
    getattr(model.features, layer_name).requires_grad = False
model_init_state = model.state_dict()

### fonction de perte & optimiser

#### def fine-tuning

In [ ]:
# pour être certain qu'on démarre avec un modèle "vide", peu importe l'ordre d'exec du notebook...
model.load_state_dict(model_init_state)
params_to_update= freeze_layers(model, n_max= 0) # toutes les couches mises à jour
#params_to_update= freeze_layers(model, n_max= n_param - 2) # tête du réseau seulement
#params_to_update= freeze_layers(model, n_max= n_param - 17-15, learning_rate= 1e-4) # Last layer
#params_to_update[-1]["lr"]= 1e-3
#params_to_update[-2]["lr"]= 1e-3

In [ ]:
learning_rate = 1e-3  # ajuster
momentum = 0.9  # à ajuster 
weight_decay = 1e-7  # ajuster
lr_step_size = 7  #ajuster 
lr_step_gamma = 0.1  # ajuster
criterion = torch.nn.CrossEntropyLoss()

#optimizer = torch.optim.SGD(filter(lambda p: p.requires_grad, model.parameters()),
#                            lr=learning_rate, momentum=momentum, weight_decay=weight_decay)
optimizer = torch.optim.SGD(params_to_update,
                            lr=learning_rate, momentum=momentum, weight_decay=weight_decay)

scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=lr_step_size, gamma=lr_step_gamma)

if use_cuda:
  model = model.cuda()

 API key https://wandb.ai/authorize

### Training

In [ ]:
%%writefile /content/.env
WANDB_API_KEY= 'morgane-magnier'

In [ ]:
from dotenv import load_dotenv
load_dotenv()

In [ ]:
!wandb login

In [ ]:
import wandb
if wandb.run is not None:
  wandb.finish()
wandb.init(project="TEL716")

# WandB – Config is a variable that holds and saves hyperparameters and inputs
config = wandb.config          # Initialize config
config.batch_size = batch_size
config.lr = learning_rate
config.momentum = momentum
config.seed = SEED
config.freeze_feature_layer_max_idx = 0
#config.log_interval = 10
# WandB – wandb.watch() automatically fetches all layer dimensions, gradients, model parameters and logs them automatically to your dashboard.
# Using log="all" log histograms of parameter values in addition to gradients
wandb.watch(model, log="all")

### Launch

In [ ]:
import wandb

epochs = 30  
train_losses, valid_losses = [], []  # pour l'affichage d'un graphe plus tard
train_accuracies, valid_accuracies = [], []  # pour l'affichage d'un graphe plus tard
best_model_state, best_model_accuracy = None, None  # pour le test final du meilleur modèle
last_print_time = time.time()
if wandb.run is not None:
  wandb.watch(model, log= 'all')
for epoch in range(epochs):

  train_loss = 0 
  train_correct, train_total = 0, 0  

  model.train()  

  for batch_idx, minibatch in enumerate(train_loader):

    if time.time() - last_print_time > 10:
      last_print_time = time.time()
      print(f"\ttrain epoch {epoch+1}/{epochs} @ iteration {batch_idx+1}/{len(train_loader)}...")

    images = minibatch[0]  # format BxCxHxW
    labels = minibatch[1]  # format Bx1
    
     # si nécessaire, on transfert nos données vers le GPU (le modèle y est déjà)
    if use_cuda:
      images = images.cuda()
      labels = labels.cuda()

    optimizer.zero_grad()

    preds = model(images)

    loss = criterion(preds, labels)

    loss.backward()

    optimizer.step()

    loss_item= loss.item()
    train_loss += loss_item  
    train_correct += (preds.topk(k=1, dim=1)[1].view(-1) == labels).nonzero().numel()
    train_total += labels.numel()
    if wandb.run is not None:
      wandb.log({
          "train_step_accuracy": 100. * train_correct / train_total,
          "train_step_Loss": loss_item})
          
  # on calcule les métriques globales pour l'epoch
  train_loss = train_loss / len(train_loader)
  train_losses.append(train_loss)
  train_accuracy = train_correct / train_total
  train_accuracies.append(train_accuracy)

  last_print_time = time.time()
  print(f"train epoch {epoch+1}/{epochs}: loss={train_loss:0.4f}, accuracy={train_accuracy:0.4f}")
  if wandb.run is not None:
    wandb.log({
        "train_epoch_accuracy": 100. * train_accuracy,
        "train_epoch_Loss": train_loss, 'epoch':epoch})

  #  'valid_loader' pour évaluer le modèle
  valid_loss = 0  # on va accumuler la perte pour afficher une courbe
  valid_correct, valid_total = 0, 0  # on va aussi accumuler les bonnes/mauvaises classifications

  model.eval()  # mise du modèle en mode "évaluation" (utile pour certaines couches...)

  # boucle semblable à celle d'entraînement, mais on utilise l'ensemble de validation
  for batch_idx, minibatch in enumerate(valid_loader):

    if time.time() - last_print_time > 10:
      last_print_time = time.time()
      print(f"\tvalid epoch {epoch+1}/{epochs} @ iteration {batch_idx+1}/{len(valid_loader)}...")

    images = minibatch[0]  
    labels = minibatch[1]  
    if use_cuda:
      images = images.cuda()
      labels = labels.cuda()

    with torch.no_grad():  # utile pour montrer explicitement qu'on n'a pas besoin des gradients
      preds = model(images)
      loss = criterion(preds, labels)

    valid_loss += loss.item()
    valid_correct += (preds.topk(k=1, dim=1)[1].view(-1) == labels).nonzero().numel()
    valid_total += labels.numel()

  # métriques globales pour l'epoch
  valid_loss = valid_loss / len(valid_loader)
  valid_losses.append(valid_loss)
  valid_accuracy = valid_correct / valid_total
  valid_accuracies.append(valid_accuracy)

  if best_model_accuracy is None or valid_accuracy > best_model_accuracy:
    best_model_state = model.state_dict()
    best_model_accuracy = valid_accuracy

  last_print_time = time.time()
  print(f"valid epoch {epoch+1}/{epochs}: loss={valid_loss:0.4f}, accuracy={valid_accuracy:0.4f}")
  print("----------------------------------------------------\n")
  if wandb.run is not None:
    wandb.log({
        "valid_accuracy": 100. * valid_accuracy,
        "valid_Loss": valid_loss})


### Final evaluation

In [ ]:
test_correct, test_total = 0, 0  # on accumule les bonnes/mauvaises classifications
model.load_state_dict(best_model_state)
model.eval()  # mise du modèle en mode "évaluation" (utile pour certaines couches...)
for minibatch in test_loader:
  images = minibatch[0]  # format BxCxHxW
  labels = minibatch[1]  # format Bx1
  if use_cuda:
    images = images.cuda()
    labels = labels.cuda()
  with torch.no_grad():
    preds = model(images)
  test_correct += (preds.topk(k=1, dim=1)[1].view(-1) == labels).nonzero().numel()
  test_total += labels.numel()
test_accuracy = test_correct / test_total
print(f"\nfinal test: accuracy={test_accuracy:0.4f}\n")

### Display validation metrics


In [ ]:
x = range(1, epochs + 1)

fig = plt.figure(figsize=(12, 4))

ax = fig.add_subplot(1, 2, 1)
ax.plot(x, train_losses, label='train')
ax.plot(x, valid_losses, label='valid')
ax.set_xlabel('# epochs')
ax.set_ylabel('Loss')
ax.legend()

ax = fig.add_subplot(1, 2, 2)
ax.plot(x, train_accuracies, label='train')
ax.plot(x, valid_accuracies, label='valid')
x_test = valid_accuracies.index(best_model_accuracy) + 1
ax.scatter(x_test, test_accuracy, color='red', label='test')
ax.set_xlabel('# epochs')
ax.set_ylabel('Accuracy')
ax.legend()

plt.show()
